In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from PIL import Image
from resizeimage import resizeimage
import cv2
import os
from IPython.display import clear_output
import matplotlib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from time import time
from sklearn.model_selection import GridSearchCV

In [25]:
X_train = np.load("X_pca.npy")
X_test = np.load("X_test_pca.npy")
X_val = np.load("X_val_pca.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")
y_val = np.load("y_val.npy")

In [3]:
svm = SVC(kernel="rbf", C=0.001, max_iter=-1)

In [4]:
svm.fit(X_train, y_train)
score = svm.score(X_test, y_test)
print(score)

0.775


In [5]:
svm = SVC(kernel = "rbf", max_iter = 1000000)
param = {"C":tuple(np.arange(0.01 - 0.002*5 + 0.0001, 0.01 + 0.002*5, 0.002))}
svm.grid = GridSearchCV(svm, param, n_jobs=1, cv=10, verbose=1, scoring="balanced_accuracy")
svm.grid.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GridSearchCV(cv=10, estimator=SVC(), n_jobs=1,
             param_grid={'C': (0.0001, 0.0021, 0.0041, 0.0061, 0.0081, 0.0101,
                               0.0121, 0.0141, 0.0161, 0.0181)},
             scoring='balanced_accuracy', verbose=1)

In [6]:
svm.grid.best_params_

{'C': 0.0181}

In [26]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [28]:
X_train, y_train = unison_shuffled_copies(X_train, y_train)
svm = SVC(kernel="rbf", C=0.01)
svm.fit(X_train, y_train)
X_test, y_test = unison_shuffled_copies(X_test, y_test)
score = svm.score(X_test, y_test)
print(score)

0.795


In [17]:
print(score)

0.78


In [15]:
len(np.arange(0.01 - 0.002*5, 0.01 + 0.002*5, 0.002))
np.arange(0.01 - 0.002*5, 0.01 + 0.002*5, 0.002)

array([0.   , 0.002, 0.004, 0.006, 0.008, 0.01 , 0.012, 0.014, 0.016,
       0.018])